In [89]:
# math and data packages
import pandas as pd
import numpy as np
import math

# charting and graphics
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

# os and file types
import os
import sys
import datetime as dt
import json
import csv

# images and display
import base64, io, IPython
from PIL import Image as PILImage
from IPython.display import Markdown as md
from IPython.display import display, Math, Latex

# import the master data
spd=pd.read_csv("resources/inprocess/master.csv")
# drop unneeded columns
spd = spd.drop(axis = 1,columns = ["Unnamed: 0","mattenstrasse-area-1"])

# remove duplicates from merging of survey databases
spd = spd[spd["locname"] != "ignore"]

In [90]:
spd["loc_date"] = list(zip(spd.locname, spd["date"]))
spd.to_csv("resources/survey_data_20_21_geo_watch_list.csv", index=False)

In [91]:
# import the master data
spd=pd.read_csv("resources/survey_data_20_21_geo_watch_list.csv")
spd["datetime"] = pd.to_datetime(spd["date"],yearfirst = True)

In [92]:
# functions that will be used in each subgroup, part of analysis and in the whole data

# number of surveys per year per location (needs loc_date column)
    
def support_genus(x):
    """Collect the genus from the genus species. Called in functions and methods that compare genus.
    :param x: df
    """
    try: 
        int_data = x.split("-")
        data = int_data[:1]
    except:
        data = "none"
    return data

def nSamplesPerYearPlace(x):
    """returns samples per year for each location
    param x: df
    """
    return x.loc_date.nunique()

def top_y_species(x,y):
    """returns list of top y number of unique species in passed df x
    :param x: df
    :param y: Integer index stop for the number of requested record
    """
    return x['species'].value_counts()[:y].index.tolist()

def top_y_genus(x,y):
    """returns list of top y number of unique genus in passed df x
    param x: df
    param y: int, final rank to include in the list
    """
    x["genus"] = x.species.map(lambda x: support_genus(x))
    return x['genus'].value_counts()[:y].index.tolist()

def top_y_group(x,y,col):
    """returns list of top y number of unique elements in column col in passed df x
    :param x: df
    :param y: int, final rank to include in list of top most common elements
    :param col: str, name of column to sort according to frequency of values
    example: top_y_group(dataframe, 10, "species") returns top 10 values in species column of provided dataframe.
    """
    return x[col].value_counts()[:y].index.tolist()

def red_list(x,y):
    """returns dataframe of species with the useful redlist codes
    of the species in passed df x in biogeo region y.
    :param x: df
    :param y: str, bioregion (ju, mp, na)
    :e.g.: {"nt": [carduus-crispus, alcea-rosea], vu = ["cardamine-hirsuta"], LC = ....}
    """
    int_data = x.loc[x["biogeo"] == y]
    for i in ["VU", "NT", "LC", "EN", "NE", "DD"]:
        if y == "ju":
            int_data.loc[int_data["rju"]==i]
        elif y == "mp":
            int_data.loc[int_data["rmp"]==i]
        else:
            int_data.loc[int_data["rch"]==i]
    return int_data
# returns a dataframe composed of the non-native status
def no_redlist_status(x):
    nix = x.loc[x['ni'] == 'ni']
    wlx = x.loc[x['inv'] == 'WL'] 
    blx = x.loc[x['inv'] == 'BL']
    int_data = nix.append(wlx)
    data = int_data.append(blx)
    return data

In [93]:
# spd.groupby(["loc_date","canton"]).biogeo.value_counts()
vals = ["mp", "ju", "na"]
lists = ["rju", "rmp", "rch"]
# wanted = {"nt": [carduus-crispus, alcea-rosea], vu = ["cardamine-hirsuta"], LC = ....}
wanted = "mp"
wanted_list = "rmp"
not_wanted = "X"

c1 = (spd.biogeo == wanted)
c2 = (spd[spd.rmp != not_wanted])

for wanted in vals:
    c1 = (spd.biogeo == wanted)
    c2 = (spd[spd.rmp != not_wanted])
    con1 = spd.loc[c1]
    con1[con1[wanted_list] != not_wanted][wanted_list].value_counts()
con1

,date,species,year,biogeo,canton,river,lot,forest,project,city,...,inv,ode,ns,ongen,ni,cdf,biel,locname,loc_date,datetime
685,2020-08-26,buddleja-davidii,1,na,be,thunersee,X,X,X,cs,...,BL,X,X,X,X,cdf,biel,sundgraben-beach,"('sundgraben-beach', '2020-08-26')",2020-08-26
686,2020-08-26,cornus-sanguinea,1,na,be,thunersee,X,X,X,cs,...,X,X,X,X,X,cdf,biel,sundgraben-beach,"('sundgraben-beach', '2020-08-26')",2020-08-26
687,2020-08-26,cotoneaster-horizontalis,1,na,be,thunersee,X,X,X,cs,...,X,X,X,X,ni,X,biel,sundgraben-beach,"('sundgraben-beach', '2020-08-26')",2020-08-26
688,2020-08-26,epilobium-angustifolium,1,na,be,thunersee,X,X,X,cs,...,X,X,X,X,X,cdf,biel,sundgraben-beach,"('sundgraben-beach', '2020-08-26')",2020-08-26
689,2020-08-26,epilobium-parviflorum,1,na,be,thunersee,X,X,X,cs,...,X,X,X,X,X,cdf,biel,sundgraben-beach,"('sundgraben-beach', '2020-08-26')",2020-08-26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5569,2021-07-22,cotoneaster-horizontalis,2,na,be,thun,X,X,renat-sund,cs,...,X,X,X,X,ni,X,biel,sundgraben-beach-1,"('sundgraben-beach-1', '2021-07-22')",2021-07-22
5570,2021-07-22,hypericum,2,na,be,thun,X,X,renat-sund,cs,...,X,X,X,ongen,X,X,X,sundgraben-beach-1,"('sundgraben-beach-1', '2021-07-22')",2021-07-22
5571,2021-07-22,linum-catharticum,2,na,be,thun,X,X,renat-sund,cs,...,X,X,X,X,X,cdf,biel,sundgraben-beach-1,"('sundgraben-beach-1', '2021-07-22')",2021-07-22
5572,2021-07-22,salix-alba,2,na,be,thun,X,X,renat-sund,cs,...,X,X,X,X,X,X,biel,sundgraben-beach-1,"('sundgraben-beach-1', '2021-07-22')",2021-07-22


In [94]:
# functions

# species per location/date combination (uniquely defines a sample)
def nSpeciesPerSample(data):
    return data.groupby(["date", "place"], as_index=False).species.count()

def aSpeciesPerSample(data, species=[]):
    """The per sample data for a species
    """
    new_data = nSpeciesPerSample(data[data["species"].isin(species)])
    new_data.rename(columns={"species":species[0]}, inplace=True)
    return new_data

# species per location
def speciesPlaceNsamps(data):
    return data.groupby(["species", "place"], as_index=False).loc_date.nunique()

def pivot_this(data, colnames, index, column, value):
    return data[colnames].pivot(index=index, columns=column, values=value).fillna(0)

def speciesPerLocation(data,colnames, index="species",columns="place",values="loc_date"):
    """Uses speciesPlaceNsamps to return a matrix of species and locations with value of
    number of times identified at that location
    """
    return pivot_this(data, colnames, index, columns, values)

def speciesPlace(species_data, soi):
    """Uses speciesPerlocation to report on one species"""
    return species_data.loc[soi, (species_data.loc[soi] > 0)]

In [95]:
spd.columns

Index(['date', 'species', 'year', 'biogeo', 'canton', 'river', 'lot', 'forest',
       'project', 'city', 'rch', 'rmp', 'rju', 'fam', 'res', 'pri', 'inv',
       'ode', 'ns', 'ongen', 'ni', 'cdf', 'biel', 'locname', 'loc_date',
       'datetime'],
      dtype='object')

In [96]:
# separate summary observations in each year

spd1 = spd.loc[spd.year == 1].copy()
spd2 = spd.loc[spd.year == 2].copy()

# top 15 genus and species in y1, y2
spd1g15 = top_y_genus(spd1,15)
spd2g15 = top_y_genus(spd2,15)
spd1sp15 = top_y_species(spd1,15)
spd2sp15 = top_y_species(spd2,15)
sp1 = spd1.species.nunique()
sp2 = spd2.species.nunique()
# top 15 non native genus and species in y1, y2

spd1nr = no_redlist_status(spd1)
spd2nr = no_redlist_status(spd2)
spd1nrg15 = top_y_genus(spd1nr,15)
spd2nrg15 = top_y_genus(spd2nr,15)
spd1nrsp15 = top_y_species(spd1nr,15)
spd2nrsp15 = top_y_species(spd2nr,15)
sp1nr = spd1nr.species.nunique()

# top 15 native fam, genus, species in y1, y2  
spd1fam = top_y_group(spd1,15,"fam")
spd2fam = top_y_group(spd2,15, "fam")
spd2ng = top_y_genus(spd2nr,15)
spd1nrg15 = top_y_species(spd1nr,15)

In [110]:
# Date summary, year 1 and year 2 basic differences

# graph unique species identified per month for year one and year two
a = spd1.groupby(by=[spd.datetime.dt.month]).species.nunique()
b = spd2.groupby(by=[spd.datetime.dt.month]).species.nunique()
c = spd1.groupby(by=[spd.datetime.dt.month]).fam.nunique()
c = spd1.groupby(by=[spd.datetime.dt.month]).fam.nunique()
# graph unique genus identified per month
c

datetime
3      6
4      5
5      2
6     10
7     14
8     47
9     32
10     4
11     5
12     2
Name: locname, dtype: int64

Year 2 saw around 5x as many plants identified as in Year 1 due to the extra experience of the surveyor in identifying plants. In Year 3, given the extra experience of the surveyor, the expected number of observations will triple to around 18'000 - 22'000 observations through the year.

Not only has experience improved plant recognition significantly, but the survey method and data pipeline are now much smoother. On top of this the processes to organize and analyze the data are now in place so the survey team is much freer to focus on identifying plants and entering them in the infoflora app.

In [99]:
# number of unique species identified and changes in top 10 genus, species, identified, red_list species, not_indigenous species
# number of instances and number of surveys
# numbers per survey
# series of monthly data, then series of quarterly data (feb-mar), (apr-may-jun),  (july-august-september-october)
spd["date"]

0       2020-09-02
1       2020-09-02
2       2020-09-02
3       2020-09-02
4       2020-09-02
           ...    
7564    2021-06-15
7565    2021-06-15
7566    2021-06-15
7567    2021-06-15
7568    2021-06-15
Name: date, Length: 7569, dtype: object

As we can see, the absolute and relative number of species identified went up as the year went on with a peak in early summer. Since the population diversity of herbaceous non grass flowering plants grows through the year and then peaks before going down again this is exactly to be expected. Notice that population of identified plants changes significantly through time as one might expect - each of the ten most common species and genus changes over time.

The second reason this is true is that the surveyor was not comfortable identifying most plants through leaves/rosettes and so species blooming later in the year would not be identified until then. This contribution to the skew in plants identified will go down in year three now that the surveyor has a strong basis in the local flora and can recognize future flowers based on leaf patterns for a significant number of the local species. If species cannot be identified, then genera can be also be more accurately noted.

An additional bias is that the surveyor collected a number of "empty" surveys of different areas with only some of the city trees in flower (such as corylus avellana) during February. This is not really an accurate represenation of the survey areas, even necessarily through the month of february and early march. As such, this data has been left out aside from a few early survey sites with some populations of flower plants. In other words, lots of empty values are left out. This will be addressed by including at least family level taxonomic identification (e.g. poaceae 1, poaceae 2, salix sp) even in places without the herbaceous vascual

The literature is clear that both late blooming and early blooming species are critically important for overwintering for small animals and especially insects. This will be taken into account to more accurately 

In [100]:
### Comparing lots, describing lots, there are four pulled out of the data here

lot_list = ["schwanen", "port", "taubenloch"]

sch = spd.loc[spd['lot'] == "schwanen"].copy()

por = spd.loc[spd['lot'] == "port"]

tau = spd.loc[spd['lot'] == "taubenloch"]

fla = spd.loc[spd["lot"] == "flaess"]


# Compare numbers in key categories (here, not indigenous, watch list, black list, near threatened in MP)
nisch = sch.loc[sch['ni'] =='ni']
wlsch = sch.loc[sch['inv'] =='WL']
blsch = sch.loc[sch['inv'] =='BL']
odesch = sch.loc[sch['ode'] == "ode"]
ntmpsch =  sch.loc[sch['rmp'] =='NT']
vumpsch =  sch.loc[sch['rmp'] =='VU']

print(F"there is {ntmpsch.species.nunique()} species on the NT list")
print(F"there is {wlsch.species.nunique()} species on the watch list")
print(F"there is {blsch.species.nunique()} species on the black list")
print(F"there is {odesch.species.nunique()} species on the ode list")
print(F"there is {vumpsch.species.nunique()} species on the vulnerable list")
print(F"there are {nisch.species.nunique()} non indigenous non invasive species")


there is 1 species on the NT list
there is 0 species on the watch list
there is 4 species on the black list
there is 0 species on the ode list
there is 0 species on the vulnerable list
there are 3 non indigenous non invasive species


In [101]:
a = top_y_species(sch,15)
c = top_y_species(sch,10)
a

['senecio-inaequidens',
 'lotus-corniculatus',
 'tussilago-farfara',
 'medicago-lupulina',
 'oenothera-biennis',
 'erigeron-annuus',
 'artemisia-vulgaris',
 'daucus-carota',
 'buddleja-davidii',
 'hypericum-perforatum',
 'senecio-vulgaris',
 'cirsium-vulgare',
 'plantago-lanceolata',
 'trifolium-repens',
 'veronica-persica']

In [102]:
d = top_y_genus(spd,15)
d

[['veronica'],
 ['trifolium'],
 ['ranunculus'],
 ['taraxacum'],
 ['geranium'],
 ['hedera'],
 ['senecio'],
 ['medicago'],
 ['galium'],
 ['plantago'],
 ['cardamine'],
 ['erigeron'],
 ['lamium'],
 ['bellis'],
 ['urtica']]

In [103]:
b = top_y_genus(sch,15)
b

[['senecio'],
 ['trifolium'],
 ['medicago'],
 ['lotus'],
 ['tussilago'],
 ['erigeron'],
 ['artemisia'],
 ['oenothera'],
 ['cirsium'],
 ['daucus'],
 ['solidago'],
 ['hypericum'],
 ['buddleja'],
 ['sonchus'],
 ['plantago']]

In [104]:
spdy = top_y_genus(spd,25)


In [105]:
spd.project.unique()

array(['X', 'renat-suze', nan, 'renat-cygnes', 'pronat', 'renat-neumatt',
       'renat-sund', 'renat-vor'], dtype=object)

In [106]:
# Describe and compare the renaturalization areas

cyg = spd.loc[spd['project'] == "renat-cygnes"]

suz = spd.loc[spd['project'] == "renat-suze"]

pro = spd.loc[spd['project'] == "pronat"]

neu = spd.loc[spd["project"] == "renat-neumatt"]

prosp = pro.species.unique()
prosp

array(['medicago-lupulina', 'erophila-verna', 'senecio-vulgaris',
       'muscari-neglectum', 'taraxacum', 'primula-acaulis',
       'glechoma-hederacea', 'vinca-minor', 'prunus-spinosa',
       'veronica-persica', 'salix-caprea', 'ranunculus-acris',
       'cardamine-flexuosa', 'lamium-purpureum',
       'capsella-bursa-pastoris', 'ranunculus-bulbosus',
       'erodium-cicutarium', 'potentilla-verna', 'veronica-hederifolia',
       'geranium-pyrenaicum', 'valerianella-locusta',
       'arabidopsis-thaliana', 'sinapis-arvensis', 'cerastium-glomeratum',
       'salvia-pratensis', 'sisymbrium-officinale', 'sanguisorba-minor',
       'reseda-lutea', 'galium-mollugo', 'urtica-dioica',
       'erigeron-annuus', 'plantago-lanceolata', 'trifolium-pratense',
       'achillea-millefolium', 'veronica-arvensis', 'myosotis-arvensis',
       'galium-aparine', 'papaver-rhoeas', 'kolkwitzia-amabilis',
       'veronica-serpyllifolia', 'prunus-serotina', 'mahonia-aquifolium',
       'geum-urbanum', 'al

In [107]:
spd['species'].value_counts()[:15].index.tolist()
lots = spd[spd["lot"] != "X"]
lots['species'].value_counts()[:15].index.tolist()

['taraxacum',
 'erigeron-annuus',
 'veronica-persica',
 'tussilago-farfara',
 'lotus-corniculatus',
 'medicago-lupulina',
 'daucus-carota',
 'plantago-lanceolata',
 'cardamine-hirsuta',
 'veronica-hederifolia',
 'bellis-perennis',
 'urtica-dioica',
 'trifolium-repens',
 'achillea-millefolium',
 'galium-mollugo']

In [ ]:
spd.river.unique()

In [ ]:
# describe and compare the rivers: madretschkanal, suze, schuessinsel, and all suze length combined vs total

suz = spd.loc[spd['river'] == "suze"]

mad = spd.loc[spd['river'] == "madretschkanal"]

sch = spd.loc[spd['river'] == "schuessinsel"]

suz.head()

In [ ]:
# describe pro nature renaturalization areas, compare to lots, to other renats, to everything else "berm"

In [ ]:
sch.species.nunique()